# Welcome to your first assignment!

Instructions are below. Please give this a try, and look in the solutions folder if you get stuck (or feel free to ask me!)

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">Just before we get to the assignment --</h2>
            <span style="color:#f71;">I thought I'd take a second to point you at this page of useful resources for the course. This includes links to all the slides.<br/>
            <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">https://edwarddonner.com/2024/11/13/llm-engineering-resources/</a><br/>
            Please keep this bookmarked, and I'll continue to add more useful links there over time.
            </span>
        </td>
    </tr>
</table>

# HOMEWORK EXERCISE ASSIGNMENT

Upgrade the day 1 project to summarize a webpage to use an Open Source model running locally via Ollama rather than OpenAI

You'll be able to use this technique for all subsequent projects if you'd prefer not to use paid APIs.

**Benefits:**
1. No API charges - open-source
2. Data doesn't leave your box

**Disadvantages:**
1. Significantly less power than Frontier Model

## Recap on installation of Ollama

Simply visit [ollama.com](https://ollama.com) and install!

Once complete, the ollama server should already be running locally.  
If you visit:  
[http://localhost:11434/](http://localhost:11434/)

You should see the message `Ollama is running`.  

If not, bring up a new Terminal (Mac) or Powershell (Windows) and enter `ollama serve`  
And in another Terminal (Mac) or Powershell (Windows), enter `ollama pull llama3.2`  
Then try [http://localhost:11434/](http://localhost:11434/) again.

If Ollama is slow on your machine, try using `llama3.2:1b` as an alternative. Run `ollama pull llama3.2:1b` from a Terminal or Powershell, and change the code below from `MODEL = "llama3.2"` to `MODEL = "llama3.2:1b"`

In [5]:
# Let's just make sure the model is loaded

!ollama pull llama3.2

pulling manifest â ‹ pulling manifest â ™ pulling manifest â ¹ pulling manifest â ¸ pulling manifest â ¼ pulling manifest â ´ pulling manifest â ¦ pulling manifest â § pulling manifest â ‡ pulling manifest 
pulling dde5aa3fc5ff...   0% â–•                â–�    0 B/2.0 GB                  pulling manifest 
pulling dde5aa3fc5ff...   0% â–•                â–�    0 B/2.0 GB                  pulling manifest 
pulling dde5aa3fc5ff...   0% â–•                â–�    0 B/2.0 GB                  pulling manifest 
pulling dde5aa3fc5ff...   0% â–•                â–�    0 B/2.0 GB                  pulling manifest 
pulling dde5aa3fc5ff...   0% â–•                â–� 2.0 MB/2.0 GB                  pulling manifest 
pulling dde5aa3fc5ff...   0% â–•                â–� 6.9 MB/2.0 GB                  pulling manifest 
pulling dde5aa3fc5ff...   0% â–•                â–� 9.9 MB/2.0 GB                  pulling manifest 
pulling dde5aa3fc5ff...   1% â–•                â–�  15 MB/2.0 GB                  pul

In [53]:
# now make sure deepseek is available via ollama

!ollama pull deepseek-r1:1.5b

pulling manifest â ‹ pulling manifest â ™ pulling manifest â ¹ pulling manifest â ¸ pulling manifest 
pulling aabd4debf0c8... 100% â–•â–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–� 1.1 GB                         
pulling 369ca498f347... 100% â–•â–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–�  387 B                         
pulling 6e4c38e1172f... 100% â–•â–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–� 1.1 KB                         
pulling f4d24e9138dd... 100% â–•â–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–�  148 B                         
pulling a85fe2a2e58e... 100% â–•â–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–�  487 B                         
verifying sha256 digest 
writing manifest 
success 


# NOW the exercise for you

Take the code from day1 and incorporate it here, to build a website summarizer that uses Llama 3.2 running locally instead of OpenAI; use either of the above approaches.

In [8]:
# imports

import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI

### create website scraping class

In [9]:
# A class to represent a Webpage
# If you're not familiar with Classes, check out the "Intermediate Python" notebook

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:

    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [10]:
# test

ed = Website("https://edwarddonner.com")
print(ed.title)
print(ed.text)

Home - Edward Donner
Home
Connect Four
Outsmart
An arena that pits LLMs against each other in a battle of diplomacy and deviousness
About
Posts
Well, hi there.
I’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (
very
amateur) and losing myself in
Hacker News
, nodding my head sagely to things I only half understand.
I’m the co-founder and CTO of
Nebula.io
. We’re applying AI to a field where it can make a massive, positive impact: helping people discover their potential and pursue their reason for being. Recruiters use our product today to source, understand, engage and manage talent. I’m previously the founder and CEO of AI startup untapt,
acquired in 2021
.
We work with groundbreaking, proprietary LLMs verticalized for talent, we’ve
patented
our matching model, and our award-winning platform has happy customers and tons of press coverage.
Connec

In [11]:
# Define our system prompt - you can experiment with this later, changing the last sentence to 'Respond in markdown in Spanish."

system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."


In [12]:
# A function that writes a User Prompt that asks for summaries of websites:

def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "\nThe contents of this website is as follows; \
please provide a short summary of this website in markdown. \
If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt

In [14]:
# test

user_prompt_for(ed)

'You are looking at a website titled Home - Edward Donner\nThe contents of this website is as follows; please provide a short summary of this website in markdown. If it includes news or announcements, then summarize these too.\n\nHome\nConnect Four\nOutsmart\nAn arena that pits LLMs against each other in a battle of diplomacy and deviousness\nAbout\nPosts\nWell, hi there.\nI’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (\nvery\namateur) and losing myself in\nHacker News\n, nodding my head sagely to things I only half understand.\nI’m the co-founder and CTO of\nNebula.io\n. We’re applying AI to a field where it can make a massive, positive impact: helping people discover their potential and pursue their reason for being. Recruiters use our product today to source, understand, engage and manage talent. I’m previously the founder and CEO of AI sta

In [15]:
def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

In [16]:
#test

messages_for(ed)

[{'role': 'system',
  'content': 'You are an assistant that analyzes the contents of a website and provides a short summary, ignoring text that might be navigation related. Respond in markdown.'},
 {'role': 'user',
  'content': 'You are looking at a website titled Home - Edward Donner\nThe contents of this website is as follows; please provide a short summary of this website in markdown. If it includes news or announcements, then summarize these too.\n\nHome\nConnect Four\nOutsmart\nAn arena that pits LLMs against each other in a battle of diplomacy and deviousness\nAbout\nPosts\nWell, hi there.\nI’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (\nvery\namateur) and losing myself in\nHacker News\n, nodding my head sagely to things I only half understand.\nI’m the co-founder and CTO of\nNebula.io\n. We’re applying AI to a field where it can make a

In [33]:
# And now: call the OpenAI API. You will get very familiar with this!


def summarize(url):
    ollama_via_openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')
    website = Website(url)
    response = ollama_via_openai.chat.completions.create(
        model=MODEL,
        messages=messages_for(website)
    ) 
    return response.choices[0].message.content

In [35]:
content = summarize("https://edwarddonner.com")

In [36]:
from IPython.display import display, Markdown
display(Markdown(content))

# Website Summary

## Introduction
The website "Home - Edward Donner" appears to be a personal webpage created by Ed, the co-founder and CTO of Nebula.io, a company applying AI to help people discover their potential.

## About Owner
Ed is a tech enthusiast who enjoys writing code, experimenting with Large Language Models (LLMs), and DJing. He has experience as a founder and CEO of ai startup untapt, which was acquired in 2021.

## Recent News and Announcements
The website's news section contains the following updates:

* **December 21, 2024**: LLM Workshop – Hands-on with Agents – resources (added to the "Connect" section)
* **November 13, 2024**: Welcome, SuperDataScientists! (added to the "Connect" section)
* **October 16, 2024**: From Software Engineer to AI Data Scientist – resources (added to the "Connect" section)
* **Newsletters and Links**: The website includes forms for subscribing to newsletters, as well as links to Ed's LinkedIn and Twitter profiles.

## Topics
The website touches on various topics related to Artificial Intelligence (AI), machine learning, and technology.

In [48]:
# move model as an input to the function

def summarize(url, model="llama3.2"):
    ollama_via_openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')

    website = Website(url)
    response = ollama_via_openai.chat.completions.create(
        model=model,
        messages=messages_for(website)
    ) 
    return response.choices[0].message.content

In [49]:
# test without an input to the model, assume model = llama3.2
content = summarize("https://edwarddonner.com")
content

'**Summary of the Website**\n==========================\n\n### Overview\n\nThe website is maintained by Edward Donner, a co-founder and CTO of Nebula.io. The platform applies AI to help people discover their potential.\n\n### News/Announcements\n\n* **LLM Workshop – Hands-on with Agents**: A workshop on December 23, 2025 (initially announced for January 23, 2025)\n* **Welcome, SuperDataScientists!** Announced November 13, 2024\n* **Mastering AI and LLM Engineering - Resources**: Released October 16, 2024\n* **From Software Engineer to AI Data Scientist – resources**: Released around September-October 2024 (date not explicitly mentioned)\n\n### Content\n\nThe website appears to be a personal blog or portfolio of the author. It includes information about his work at Nebula.io and untapt, as well as his interests in DJing, electronic music production, and Hacker News.\n \nNote that the date format used on this webpage is inconsistent as one month uses standard date format, but some months

In [50]:
from IPython.display import display, Markdown
display(Markdown(content))

**Summary of the Website**
==========================

### Overview

The website is maintained by Edward Donner, a co-founder and CTO of Nebula.io. The platform applies AI to help people discover their potential.

### News/Announcements

* **LLM Workshop – Hands-on with Agents**: A workshop on December 23, 2025 (initially announced for January 23, 2025)
* **Welcome, SuperDataScientists!** Announced November 13, 2024
* **Mastering AI and LLM Engineering - Resources**: Released October 16, 2024
* **From Software Engineer to AI Data Scientist – resources**: Released around September-October 2024 (date not explicitly mentioned)

### Content

The website appears to be a personal blog or portfolio of the author. It includes information about his work at Nebula.io and untapt, as well as his interests in DJing, electronic music production, and Hacker News.
 
Note that the date format used on this webpage is inconsistent as one month uses standard date format, but some months have different abbreviations (e.g., 12/21, which is written out more fully as December 21)

In [51]:
# testing with deepdeak
model="deepseek-r1:1.5b"
content = summarize("https://edwarddonner.com", model)
content

'<think>\nAlright, so I\'m trying to come up with a short summary for this website. Let me break it down step by step.\n\nFirst, the site\'s name is "Home - Edward Donner," which suggests it\'s probably dedicated to something related to his field. He\'s an AI scientist or someone in tech, given his mentions of LLMs and AI engineering.\n\nLooking at what\'s listed under Home: Connect Four, Outsmart, and another arena for LLMs. I know that "Connect Four" is a board game with strategic aspects, so maybe it\'s about developing strategies or algorithms, which would tie into the LLM aspect if those are part of his work.\n\n"Outsmart" sounds like a game where you try to outmaneuver opponents; perhaps strategies used in AI applications. The last point mentions fighting a large positive impact on people\'s lives using technology, which ties back to Edward\'s role as an innovator and entrepreneur.\n\nOn About, he talks about writing code and experimenting with LLMs. He mentions his co-founders a

In [52]:
from IPython.display import display, Markdown
display(Markdown(content))

<think>
Alright, so I'm trying to come up with a short summary for this website. Let me break it down step by step.

First, the site's name is "Home - Edward Donner," which suggests it's probably dedicated to something related to his field. He's an AI scientist or someone in tech, given his mentions of LLMs and AI engineering.

Looking at what's listed under Home: Connect Four, Outsmart, and another arena for LLMs. I know that "Connect Four" is a board game with strategic aspects, so maybe it's about developing strategies or algorithms, which would tie into the LLM aspect if those are part of his work.

"Outsmart" sounds like a game where you try to outmaneuver opponents; perhaps strategies used in AI applications. The last point mentions fighting a large positive impact on people's lives using technology, which ties back to Edward's role as an innovator and entrepreneur.

On About, he talks about writing code and experimenting with LLMs. He mentions his co-founders and CTO, Nebula.io, which is part of their own story. So this is definitely a tech company focused on AI applications in talent management or development.

posts list several key events: LLM workshops, welcome to SuperDataScients, Mastering AI engineering resources, job descriptions from software engineer to AI data scientist roles, and his open email subscription info, including links to LinkedIn, Twitter, Facebook, etc. These seem like events that would be hosted on a company website or internal blog.

Navigation points go back to Connect Four, Outsmart, LLM arena, About section, Posts, and the social media links. The social media tabs are for LinkedIn, Twitter, Facebook, which are standard for tech companies, maybe more specifically in AI contexts.

Putting this all together, Edward's company is probably focused on developing tools that help people discover their potential through technology, with a specific interest in machine learning applications such as talent management and job placement. The site provides an overview of their products (Connect Four, Outsmart) and events related to LLMs, serving both the company and tech enthusiasts.

I should make sure I include all the main points without going into too much detail but still cover essential sections like history, services, press, what's new, navigation, and social links. Also, note that while it mainly includes news about them, they cater to tech geeks specifically in fields where LLMs can have a positive impact.
</think>

** Edward Donner Tech Hub**

Edward Donner leads the development of a tech-driven company dedicated to enhancing people's potential through innovative technology. Focused on applications like machine learning for talent management and job placement, his work is aimed at creating tools that aid individuals in discovering their value and succeeding.

**Services:**
- **Connect Four:** Strategic thinking algorithms developed by Edward.
- **Outsmart:** Algorithms designed to outmaneuver opponents in AI-related strategies.

**Events and News:**
- **LLM Workshops:** Hands-on agent training with resources.
- **Welcome to SuperDataScients:** Initial launch event.
- **AI Engineering Resources:** Comprehensive guides and tools.
- **Talent Job Descriptions (SL) - 13 Oct 2024:** Job listings from software engineer to AI data scientist roles.

**Navigation:**
- Connect Four, Outsmart, LLM Arena: Key development areas.
- About section: Background on Edward and his impact in tech.
- Post list: Events highlights for tech enthusiasts.

**Social Media Links:**
For more updates and insights, visit LinkedIn for professional networks, Twitter for real-time news, Facebook for community discussions. The site serves both as a platform for the company and an avenue for tech enthusiasts exploring EDW's products.

This hub serves as Edward Donner's tech hub, positioning him as a leader in AI applications within talent management, with a mission to inspire individuals through innovative technology.

In [ ]:
# now create a gui to select the model and the website
# let's use ipywidgets


In [59]:
from ipywidgets import interact

import ipywidgets as widgets
from IPython.display import display

# Function to summarize website using input
def summarize_website_and_print(model_dropdown_value, website_textbox_value):
    print(f"Dropdown value: {model_dropdown_value}, Textbox value: {website_textbox_value}")
    content = summarize(website_textbox_value, model_dropdown_value)
    print(content)

# Create a dropdown widget
model_dropdown = widgets.Dropdown(
    options=["llama3.2", "deepseek-r1:1.5b"],
    description='Select an model:'
)

# Create a textbox to enter the website
website_textbox = widgets.Text(placeholder='https://edwarddonner.com', description='Website to Summarize:',value="https://edwarddonner.com")

# Create a button widget
button = widgets.Button(description="Summarize website")

# Function to call when the button is clicked
def on_button_clicked(b):
    summarize_website_and_print(model_dropdown_value=model_dropdown.value, website_textbox_value=website_textbox.value)

# Link the button click event to the function
button.on_click(on_button_clicked)

# Display the widgets
ui = widgets.VBox([model_dropdown, website_textbox, button])
display(ui)


In [ ]:
import ipywidgets as widgets
from IPython.display import display

# Define the function to be called
def my_function(dropdown_value, textbox_value):
    print(f"Dropdown value: {dropdown_value}, Textbox value: {textbox_value}")

# Create widgets
dropdown = widgets.Dropdown(options=['Option 1', 'Option 2', 'Option 3'], description='Select:')
textbox = widgets.Text(placeholder='Enter text here', description='Text:')
button = widgets.Button(description='Run')

# Define button click event handler
def on_button_clicked(b):
    my_function(dropdown.value, textbox.value)

# Attach the event handler to the button
button.on_click(on_button_clicked)

# Display the widgets
ui = widgets.VBox([dropdown, textbox, button])
display(ui)

In [ ]:
import ipywidgets as widgets



def my_function(dropdown_value, text_value):

    print(f"Dropdown selected: {dropdown_value}, Textbox value: {text_value}") 



# Create widgets

dropdown = widgets.Dropdown(options=['Option 1', 'Option 2', 'Option 3'], value='Option 1', description='Select Option:')

textbox = widgets.Text(value='Default Text', description='Enter Text:')

button = widgets.Button(description='Run Function')



# Connect button click to function call

def on_button_click(b):

    my_function(dropdown.value, textbox.value)

button.on_click(on_button_click)



# Display the interactive UI

widgets.interact(my_function, dropdown=dropdown, text_value=textbox, button=button)
